In [2]:
%matplotlib inline
import collections
import random
import re
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

### 文本预处理

In [20]:
#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt','090b5e7e70c295757f55df93cb0a180b9691891a')
def read_time_machine(): #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

In [21]:
 #词元化
def tokenize(lines, token='word'): #@save
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

In [5]:
#词表
"""
词表（vocabulary），用来将字符串类型的词元映射到从0开始的数字索引中。
1.先将训练集中的所有文档合并在一起，对它们的唯一词元进行统计，得到的统计结果称之为语料（corpus）
2.然后根据每个唯一词元的出现频率，为其分配一个数字索引。很少出现的词元通常被移除，这可以降低复杂性。
3.另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“<unk>”。
"""
def count_corpus(tokens):
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list): # tookens内内嵌列表，
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

class Vocab:
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens) # 生成元组，内有词元和其频率
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True) #[(词元，频率),()]
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                #为词元添加索引，其结构{token: idx,},其以预定词元开头
                self.token_to_idx[token] = len(self.idx_to_token) - 1
                
    def __len__(self):
        return len(self.idx_to_token)

    #通过token获取索引
    def __getitem__(self, tokens):
        # 当传入的 tokens 参数不是列表或元组时（即假设它是单个令牌），方法会尝试从一个名为 self.token_to_idx 的字典中获取该令牌对应的索引值。如果令牌不存在于字典中，方法将返回一个未知令牌的索引，这通常是一个预定义的属性 self.unk。
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        # 如果传入的 tokens 参数是一个列表或元组（即一组令牌），方法将遍历集合中的每个令牌，对每个令牌递归调用 __getitem__ 方法，并将结果收集成一个列表返回
        return [self.__getitem__(token) for token in tokens]

    # 通过索引获取token
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property #unk 属性被定义为一个装饰器 @property，这意味着它可以像访问数据属性一样被调用，而不需要括号。
    def unk(self): # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

In [22]:
lines = read_time_machine() #[[第一行文字],[第二行文字],.....]
tokens = tokenize(lines)#[[第一行tokens,[第二二行tokens]]]
vocab = Vocab(tokens)
#将文本转为数字索引.索引转文本
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])
    print('索引转文本:', vocab.to_tokens(vocab[tokens[i]]))

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
索引转文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]
索引转文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [29]:
#整合功能
def load_corpus_time_machine(max_tokens=-1): #@save
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line] #文本列表转成索引列表
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab
# corpus 是文本经过tokenlize后的文本索引
corpus, vocab = load_corpus_time_machine()

### 读取长序列数据 

In [43]:
#随机采样
"""在随机采样中，每个样本都是在原始的长序列上任意捕获的子序列。在迭代过程中，来自两个相邻的、随机
的、小批量中的子序列不一定在原始序列上相邻。对于语言建模，目标是基于到目前为止我们看到的词元来
预测下一个词元，因此标签是移位了一个词元的原始序列。"""
def seq_data_iter_random(corpus, batch_size, num_steps):
    """使用随机抽样生成一个小批量子序列"""
    # 从随机偏移量开始对序列进行分区，随机范围包括num_steps-1
    print(f"corpus:{corpus}\nbatch_size:{batch_size}\nnum_steps:{num_steps}")
    corpus = corpus[random.randint(0, num_steps - 1): ]
    print(f"corpus:{corpus}\nlen(corpus):{len(corpus)}")
    # 减去1，是因为我们需要考虑标签
    num_subseqs = (len(corpus) - 1) // num_steps
    print(f"num_subseqs:{num_subseqs}")
    # 长度为num_steps的子序列的起始索引
    initial_indices = list(range(0, num_subseqs * num_steps, num_steps))
    print(f"initial_indices:{initial_indices}")
    # 在随机抽样的迭代过程中，
    # 来自两个相邻的、随机的、小批量中的子序列不一定在原始序列上相邻
    random.shuffle(initial_indices)
    print(f"打乱后initial_indices:{initial_indices}")
    def data(pos):
        # 返回从pos位置开始的长度为num_steps的序列
        return corpus[pos: pos + num_steps]
    num_batches = num_subseqs // batch_size
    print(f"num_batches:{num_batches}")
    for i in range(0, batch_size * num_batches, batch_size):
        # 在这里，initial_indices包含子序列的随机起始索引
        initial_indices_per_batch = initial_indices[i: i + batch_size]
        print(f"initial_indices_per_batch:{initial_indices_per_batch}")
        X = [data(j) for j in initial_indices_per_batch]
        Y = [data(j + 1) for j in initial_indices_per_batch]
        yield torch.tensor(X), torch.tensor(Y)

In [44]:
my_seq = corpus[0:35]
my_char = vocab.to_tokens(my_seq)
print(my_char)
for X, Y in seq_data_iter_random(my_seq, batch_size=2, num_steps=5):
    print('X: ', X, '\nY:', Y)

['t', 'h', 'e', ' ', 't', 'i', 'm', 'e', ' ', 'm', 'a', 'c', 'h', 'i', 'n', 'e', ' ', 'b', 'y', ' ', 'h', ' ', 'g', ' ', 'w', 'e', 'l', 'l', 's', 'i', 't', 'h', 'e', ' ', 't']
corpus:[3, 9, 2, 1, 3, 5, 13, 2, 1, 13, 4, 15, 9, 5, 6, 2, 1, 21, 19, 1, 9, 1, 18, 1, 17, 2, 12, 12, 8, 5, 3, 9, 2, 1, 3]
batch_size:2
num_steps:5
corpus:[3, 5, 13, 2, 1, 13, 4, 15, 9, 5, 6, 2, 1, 21, 19, 1, 9, 1, 18, 1, 17, 2, 12, 12, 8, 5, 3, 9, 2, 1, 3]
len(corpus):31
num_subseqs:6
initial_indices:[0, 5, 10, 15, 20, 25]
打乱后initial_indices:[5, 10, 0, 15, 20, 25]
num_batches:3
initial_indices_per_batch:[5, 10]
X:  tensor([[13,  4, 15,  9,  5],
        [ 6,  2,  1, 21, 19]]) 
Y: tensor([[ 4, 15,  9,  5,  6],
        [ 2,  1, 21, 19,  1]])
initial_indices_per_batch:[0, 15]
X:  tensor([[ 3,  5, 13,  2,  1],
        [ 1,  9,  1, 18,  1]]) 
Y: tensor([[ 5, 13,  2,  1, 13],
        [ 9,  1, 18,  1, 17]])
initial_indices_per_batch:[20, 25]
X:  tensor([[17,  2, 12, 12,  8],
        [ 5,  3,  9,  2,  1]]) 
Y: tensor([[ 2

In [10]:
#顺序分区
"""在迭代过程中，除了对原始序列可以随机抽样外，我们还可以保证两个相邻的小批量中的子序列在原始序列
上也是相邻的。这种策略在基于小批量的迭代过程中保留了拆分的子序列的顺序，因此称为顺序分区。"""
def seq_data_iter_sequential(corpus, batch_size, num_steps): #@save
    """使用顺序分区生成一个小批量子序列"""
    # 从随机偏移量开始划分序列
    offset = random.randint(0, num_steps)
    num_tokens = ((len(corpus) - offset - 1) // batch_size) * batch_size
    print(f"num_tokens:{num_tokens}")
    Xs = torch.tensor(corpus[offset: offset + num_tokens])
    Ys = torch.tensor(corpus[offset + 1: offset + 1 + num_tokens])
    Xs, Ys = Xs.reshape(batch_size, -1), Ys.reshape(batch_size, -1)
    num_batches = Xs.shape[1] // num_steps
    for i in range(0, num_steps * num_batches, num_steps):
        X = Xs[:, i: i + num_steps]
        Y = Ys[:, i: i + num_steps]
        yield X, Y


In [12]:
class SeqDataLoader: #@save
    """加载序列数据的迭代器"""
    def __init__(self, batch_size, num_steps, use_random_iter, max_tokens):
        if use_random_iter:
            self.data_iter_fn = d2l.seq_data_iter_random
        else:
            self.data_iter_fn = d2l.seq_data_iter_sequential
        self.corpus, self.vocab = d2l.load_corpus_time_machine(max_tokens)
        self.batch_size, self.num_steps = batch_size, num_steps
    def __iter__(self):
        return self.data_iter_fn(self.corpus, self.batch_size, self.num_steps)
        
def load_data_time_machine(batch_size, num_steps, use_random_iter=False, max_tokens=10000):
    """返回时光机器数据集的迭代器和词表"""
    data_iter = SeqDataLoader(batch_size, num_steps, use_random_iter, max_tokens)
    return data_iter, data_iter.vocab

### RNN

In [13]:
batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

### 初始化模型 

In [15]:
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size
    
    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01

    #隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))
    W_hh = normal((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens, device=device)
    #输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    #附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

### 循环神经网络模型 

In [50]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device),)
def rnn(inputs, state, params):
    #inputs的形状：（时间步， 批量大小， 词表大小）
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    #x的形状：（批量大小， 词表大小）
    for X in inputs:
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)

In [48]:
class RNNModelScratch: #@save
    """从零开始实现的循环神经网络模型"""
    def __init__(self, vocab_size, num_hiddens, device, get_params, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn
        
    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)
        
    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [46]:
def predict_ch8(prefix, num_preds, net, vocab, device): #@save
    """在prefix后面生成新字符"""
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    for y in prefix[1:]: # 预热期
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds): # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [51]:
predict_ch8('time traveller ', 10, net, vocab, d2l.try_gpu())

batch_size:1
num_hiddens:512
init_rnn_staee shape: (1, 512)
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
vocab_size:28
input_X(独热编码后):torch.Size([1, 1, 28])
voca

'time traveller yiohf orwg'